In [2]:
from gpaw import GPAW, PW, Davidson, FermiDirac
from gpaw import restart
from gpaw.mpi import size, rank
from ase.build import fcc111
from ase.build import bulk
from ase.io import Trajectory
from ase.optimize import BFGS
import os

In [ ]:
# Set up the bulk copper crystal structure
a = 4.18462
atoms = bulk('Au', 'fcc', a = a)

# Set up the surface slab
slab = fcc111('Au', size = (3, 3, 3), a = a, vacuum = 6.0)

# Set up the calculator
ecut = 450.0  # eV
kpts = (4, 4, 1)
calc = GPAW(mode = PW(ecut), xc = 'PBE', kpts = kpts, txt = f'T3_Au.txt')

# Relax the surface
slab.set_calculator(calc)
dyn = BFGS(slab, trajectory='relax.traj')
dyn.run(fmax = 0.05)

# Calculate the surface energy
energy_bulk = atoms.get_potential_energy() / len(atoms)
energy_slab = slab.get_potential_energy()
natoms_slab = len(slab)
natoms_bulk = len(atoms)
surface_area = 2 * (slab.cell[0, 0] * slab.cell[1, 1])
surface_energy = (energy_slab - natoms_slab / natoms_bulk * energy_bulk) / surface_area

print("Surface energy: %f J/m^2" % surface_energy)


In [7]:
from ase.visualize import view

In [16]:
a = 4.18462
atoms = bulk('Au', 'fcc', a = a)
natoms_bulk = len(atoms)
slab = fcc111('Au', size = (3, 3, 3), a = a, vacuum = 6.0)
natoms_slab = len(slab)

print(natoms_bulk, natoms_slab)
view(slab)
print(2 * (slab.cell[0, 0] * slab.cell[1, 1])) # This seems about right.
print(slab.cell[0, 0], slab.cell[1, 1])
print(slab.cell)

1 27
136.4850848002617
8.876919536066552 7.687637825584005
Cell([[8.876919536066552, 0.0, 0.0], [4.438459768033276, 7.687637825584005, 0.0], [0.0, 0.0, 16.831982966912584]])


In [ ]:
from ase.db import connect
from ase.build import fcc111
from ase.optimize import BFGS
from gpaw import GPAW, PW

db1 = connect('bulk.db')
db2 = connect('ads.db')

def run(symb, a, row):
    slab = fcc111(symb, (3, 3, 3), a=a, vacuum = 6)
    # Calculator:
    calc = GPAW(xc   = 'PBE', 
                mode = PW(450),
                kpts = [4, 4, 1],
                txt  = f'refs.txt')
    slab.set_calculator(calc)
    # Relaxation:
    dyn = BFGS(slab)
    dyn.run(fmax = 0.1)

    # Calculate the surface energy
    natoms_slab    = len(slab)
    surface_area   = 2 * (slab.cell[0, 0] * slab.cell[1, 1])

    energy_slab    = slab.get_potential_energy() # Should also print this energy. Energy of the surface != Surface energy!
    energy_bulk    = row.energy # For 1 atom.

    surface_energy = (energy_slab - natoms_slab * energy_bulk) / surface_area
    surface_energy_tot = (energy_slab - natoms_slab * energy_bulk) 
    print("Surface energy: %f eV/\AA^2" % surface_energy) 
    print("Total surface energy: %f eV" % surface_energy_tot)

    return atoms

# Clean slabs:
for row in db1.select():
    a = row.a
    symb = row.symbols[0]
    id = db2.reserve(surf=symb, ads='clean')
    if id is not None:
        atoms = run(symb, a, row)
        db2.write(atoms, id=id, surf=symb, ads='clean')

In [24]:
from ase.db import connect
# Tghis works fine...
db1 = connect('../Task6/ads.db')
for row in db1.select():
    a = row.height; print(a)

1.3421853179047378
2.0
1.253492201203878
2.0
1.3097094490914412
2.0
